In [1]:
# !pip3 install openpyxl

In [6]:
# Loading packages

import pandas as pd
import numpy as np

import plotly.express as px

In [3]:
data_price = pd.read_csv('./src/get_around_pricing_project.csv', index_col=[0])
data_price.head()

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


In [4]:
data_delay = pd.read_excel('./src/get_around_delay_analysis.xlsx')
data_delay.head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN


# EDA

## Price

In [5]:
data_price.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4843 entries, 0 to 4842
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   model_key                  4843 non-null   object
 1   mileage                    4843 non-null   int64 
 2   engine_power               4843 non-null   int64 
 3   fuel                       4843 non-null   object
 4   paint_color                4843 non-null   object
 5   car_type                   4843 non-null   object
 6   private_parking_available  4843 non-null   bool  
 7   has_gps                    4843 non-null   bool  
 8   has_air_conditioning       4843 non-null   bool  
 9   automatic_car              4843 non-null   bool  
 10  has_getaround_connect      4843 non-null   bool  
 11  has_speed_regulator        4843 non-null   bool  
 12  winter_tires               4843 non-null   bool  
 13  rental_price_per_day       4843 non-null   int64 
dtypes: bool(7), i

No missing values, all data are of the correct type

In [6]:
# Only for quantitative data
data_price.describe()

,mileage,engine_power,rental_price_per_day
count,4.843000e+03,4843.00000,4843.000000
mean,1.409628e+05,128.98823,121.214536
std,6.019674e+04,38.99336,33.568268
min,-6.400000e+01,0.00000,10.000000
25%,1.029135e+05,100.00000,104.000000
50%,1.410800e+05,120.00000,119.000000
75%,1.751955e+05,135.00000,136.000000
max,1.000376e+06,423.00000,422.000000


minimum mileage is negative ? Maximum value > 1 Million km ! -> likely fat finger issue

In [7]:
px.box(data_price, x='mileage')

In [8]:
neg_mile = data_price['mileage'] < 0 
data_price[neg_mile]

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
2938,Renault,-64,230,diesel,black,sedan,True,True,False,True,False,False,True,274


Only one car has issue in negative mileage

In [9]:
mil_mile = data_price['mileage'] > 1000000
data_price[mil_mile]

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
3732,Citroën,1000376,90,diesel,black,subcompact,True,False,False,False,False,False,True,37


Only one car has issue in unlimited mileage

**We should just drop these two lines**

In [10]:
data_price = data_price[~neg_mile & ~mil_mile]

In [11]:
# We dropped the two outliers
len(data_price)

4841

## Some features that may be associated with delays in returning the car or not :

In [12]:
px.pie(data_price, 'has_getaround_connect', title='has_getaround_connect', color='has_getaround_connect', color_discrete_map={True: 'blue', False: 'red'})

In [13]:
px.pie(data_price, 'private_parking_available', title='private_parking_available', color='private_parking_available', color_discrete_map={True: 'blue', False: 'red'})

In [14]:
px.pie(data_price, 'has_gps', title='has_gps', color='has_gps', color_discrete_map={True: 'blue', False: 'red'})


## Delay

In [15]:
data_delay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21310 entries, 0 to 21309
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rental_id                                   21310 non-null  int64  
 1   car_id                                      21310 non-null  int64  
 2   checkin_type                                21310 non-null  object 
 3   state                                       21310 non-null  object 
 4   delay_at_checkout_in_minutes                16346 non-null  float64
 5   previous_ended_rental_id                    1841 non-null   float64
 6   time_delta_with_previous_rental_in_minutes  1841 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 1.1+ MB


Some missing values in the delay informations. 'previous ended rental id' is of type float whereas it should be int 64 as in 'rental id'? 

### Investigating float id

In [16]:
data_delay_temp = data_delay['previous_ended_rental_id'].dropna()

In [17]:
(data_delay_temp != data_delay_temp.astype(int)).sum()

0

So actually, there are no ID as float, but the dataframe column was most likely converted to float as it includes NA, but the IDs are all int

### Investigating NA

In [18]:
def na_per(df):
    ''' will calculate missing data in percentage for each column in the table'''
    tot = len(df)
    for c in df.columns:
        na = df[c].isna().sum()
        if na > 0:
            percentage_missing=(na/tot)*100
            print(f'{percentage_missing:.2f}% of data is missing in the column \'{c}\'.')
        else : 
            print(f'No missing value in the column \'{c}\'.')

In [19]:
na_per(data_delay)

No missing value in the column 'rental_id'.
No missing value in the column 'car_id'.
No missing value in the column 'checkin_type'.
No missing value in the column 'state'.
23.29% of data is missing in the column 'delay_at_checkout_in_minutes'.
91.36% of data is missing in the column 'previous_ended_rental_id'.
91.36% of data is missing in the column 'time_delta_with_previous_rental_in_minutes'.


From the data_delay doc : 

1) delay_at_checkout_in_minutes = Difference in minutes between the rental end time requested by the driver when booking the car and the actual time the driver completed the checkout. Negative values mean that the driver returned the car in advance.

2) previous_ended_rental_id = id of the previous ended rental of the car (NULL when no previous rental or delay with previous rental higher than 12 hours)


3) time_delta_with_previous_rental_in_minutes = Difference in minutes between this rental planned start time and the previous rental planned end time (when lower than 12 hours, NULL if higher)

**So NA in 2 and 3 indicate cases where the client did not face the mentionned issue in the project (high friction between previous and next drivers)**

## **So we should work only with the data from the remaining 8.64% where no NA in those two columns !**


In [20]:
data_delay.describe()

,rental_id,car_id,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.000000,21310.000000,16346.000000,1841.000000,1841.000000
mean,549712.880338,350030.603426,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,67.000000,560823.000000,540.000000
max,576401.000000,417675.000000,71084.000000,575053.000000,720.000000


It seems that some values in minutes are aberrant in 'delay_at_checkout' (see min max which are thousands of hours when converted). Let's convert in hour and filter for + or - 12 hours at max

In [21]:
def convert_m_to_h(data, col_name: str):
    ''' Will convert a minute column in hours '''
    new_col = f'{col_name}_converted_in_hours'
    data[new_col] = data[col_name]/60

In [22]:
convert_m_to_h(data_delay, 'delay_at_checkout_in_minutes')
data_delay.head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay_at_checkout_in_minutes_converted_in_hours
0,505000,363965,mobile,canceled,NaN,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN,-1.350000
2,508131,359049,connect,ended,70.0,NaN,NaN,1.166667
3,508865,299063,connect,canceled,NaN,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN,NaN


In [23]:
convert_m_to_h(data_delay, 'time_delta_with_previous_rental_in_minutes')
data_delay.head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay_at_checkout_in_minutes_converted_in_hours,time_delta_with_previous_rental_in_minutes_converted_in_hours
0,505000,363965,mobile,canceled,NaN,NaN,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN,-1.350000,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN,1.166667,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN,NaN,NaN


In [24]:
data_delay.describe()

,rental_id,car_id,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay_at_checkout_in_minutes_converted_in_hours,time_delta_with_previous_rental_in_minutes_converted_in_hours
count,21310.000000,21310.000000,16346.000000,1841.000000,1841.000000,16346.000000,1841.000000
mean,549712.880338,350030.603426,59.701517,550127.411733,279.288430,0.995025,4.654807
std,13863.446964,58206.249765,1002.561635,13184.023111,254.594486,16.709361,4.243241
min,504806.000000,159250.000000,-22433.000000,505628.000000,0.000000,-373.883333,0.000000
25%,540613.250000,317639.000000,-36.000000,540896.000000,60.000000,-0.600000,1.000000
50%,550350.000000,368717.000000,9.000000,550567.000000,180.000000,0.150000,3.000000
75%,560468.500000,394928.000000,67.000000,560823.000000,540.000000,1.116667,9.000000
max,576401.000000,417675.000000,71084.000000,575053.000000,720.000000,1184.733333,12.000000


**This filter will retain only delays within a 12h window AND remove NA for the delay_at_checkout columns (minutes and hours) (which were of 21%)**


In [25]:
twelvecond = data_delay['delay_at_checkout_in_minutes_converted_in_hours'].between(-12, 12)
data_delay_trim = data_delay[twelvecond]
data_delay_trim.isna().sum()

rental_id                                                            0
car_id                                                               0
checkin_type                                                         0
state                                                                0
delay_at_checkout_in_minutes                                         0
previous_ended_rental_id                                         14148
time_delta_with_previous_rental_in_minutes                       14148
delay_at_checkout_in_minutes_converted_in_hours                      0
time_delta_with_previous_rental_in_minutes_converted_in_hours    14148
dtype: int64

In [26]:
len(data_delay_trim)

15608

In [27]:
data_delay_trim['state'].value_counts()

state
ended    15608
Name: count, dtype: int64

State reflects only the observation that all vehicles returned in advance were cancelled. Not relevant to our question.

In [28]:
hist_delay = px.histogram(data_delay_trim, y='delay_at_checkout_in_minutes_converted_in_hours')
hist_delay.add_hline(y=0, line_color='red')
hist_delay.show()

Only the positive values are late drivers.

In [29]:
delay_late = data_delay_trim['delay_at_checkout_in_minutes'] > 0
data_late = data_delay_trim[delay_late]

In [30]:
len(data_late) / len(data_delay_trim) *100

57.08610968733983

**57% of drivers were late !**

In [31]:
px.histogram(data_late, 'delay_at_checkout_in_minutes_converted_in_hours', color='checkin_type', color_discrete_map={'mobile': 'lightblue', 'connect': 'purple'})

**Most delays occurs in a ~ 2 hours window !**

We saw earlier that the **connect** feature was present in ~50% but most people uses mobile. Here, we would be tempted to say that there are more people late using the mobile app than the connect feature but it is the trend that matter the most, and they are rather similar. 

# Threshold: how long should the minimum delay be?

In [32]:
data_late['delay_at_checkout_in_minutes_converted_in_hours'].mean()

1.5077085671530117

**Late drivers are ~ 1.5 hours late**. This could act as a **threshold**, but should be compared to potential loss for Getaround. Unfortunately, no key that would allow to join the two dataframe. To investigate later !

Ok so we have a percentage of late driver, but only a fraction of them triggered friction with the subsequent rental of the same car. We should isolate case where friction occured among all late car return and see how it compares with a potential 1.5h threshold

the 'time_delta_with_previous_rental' indicates the typical turnover window between rentals of the same car. If the mean 'delay at checkout' is < mean 'time_delta', it would indicate that 1.5 threshold is ok.

In [33]:
data_late['time_delta_with_previous_rental_in_minutes_converted_in_hours'].mean()

4.296488946684005

In [34]:
px.box(data_late, 'time_delta_with_previous_rental_in_minutes_converted_in_hours')

In [35]:
data_late['time_delta_with_previous_rental_in_minutes_converted_in_hours'].median()

2.5

### The 1.5h threshold is inferior to the mean (4.3h) and median (2.5h) turnover between rentals for the same car. So it would seem that 1.5 would be ok. Though it is very long and the calculation was quite simple. Let's try a more granular approach now that we have a baselin idea.

If the time delta represents the available time of a car between two rentals, and the delay at checkout represent the late amount in min/hours, then there will be friction only when **delta < delay**. So could calculate that for all available data.

Unfortunately, for one rental id, we cannot compare the delta and the delay because the delta is clculated based on the end time from the previous rental. We could have only calculated friction per car if delta was based on the end time of the actual rental and the start time of the next one. So we cannot just check if delta < delay for each row, because it would be wrong (although it could still be statistically informative)

One way to look at the amount of friction would be to investigate in how many cases the **threshold** would be **< delay** (indicating an insufficient threshold to circumvent new users waiting for the previous users who is late)

In [276]:
def friction(threshold):
    '''Will calculate the number of problematic cases (friction) remaining among late drivers after defining threshold '''

    friction = threshold < data_late['delay_at_checkout_in_minutes_converted_in_hours']
    nb_prob = friction.sum()
    late_percentage = nb_prob/len(data_late)
    print(f'There was {late_percentage*100:.2f}% of friction ({nb_prob} problematic cases) corrected with a {threshold}h threshold.')
    return nb_prob, late_percentage

In [280]:
nb_prob, late_percentage = friction(1.5)

There was 31.01% of friction (2763 problematic cases) corrected with a 1.5h threshold.


### So even by preventing new users to rent a car during 1.5h after its requested end time from the previous users, we still have a 31% level of friction among late users. 

In [234]:
# 0.5 h is 30 min

fric_dic = {}
for i in np.arange(0, 12, 0.5):
    fric_dic[i] = friction(i)

There was 100.00% of friction with a 0.0h threshold.
There was 62.67% of friction with a 0.5h threshold.
There was 43.68% of friction with a 1.0h threshold.
There was 31.01% of friction with a 1.5h threshold.
There was 23.11% of friction with a 2.0h threshold.
There was 17.68% of friction with a 2.5h threshold.
There was 14.16% of friction with a 3.0h threshold.
There was 11.31% of friction with a 3.5h threshold.
There was 9.43% of friction with a 4.0h threshold.
There was 7.45% of friction with a 4.5h threshold.
There was 6.23% of friction with a 5.0h threshold.
There was 5.24% of friction with a 5.5h threshold.
There was 4.39% of friction with a 6.0h threshold.
There was 3.80% of friction with a 6.5h threshold.
There was 3.21% of friction with a 7.0h threshold.
There was 2.66% of friction with a 7.5h threshold.
There was 2.14% of friction with a 8.0h threshold.
There was 1.83% of friction with a 8.5h threshold.
There was 1.44% of friction with a 9.0h threshold.
There was 1.09% of fri

In [235]:
fric_dic

{0.0: 100.0,
 0.5: 62.67115600448933,
 1.0: 43.681257014590344,
 1.5: 31.01010101010101,
 2.0: 23.108866442199773,
 2.5: 17.67676767676768,
 3.0: 14.163860830527497,
 3.5: 11.313131313131313,
 4.0: 9.427609427609427,
 4.5: 7.45230078563412,
 5.0: 6.228956228956229,
 5.5: 5.241301907968575,
 6.0: 4.388327721661055,
 6.5: 3.8047138047138045,
 7.0: 3.2098765432098766,
 7.5: 2.65993265993266,
 8.0: 2.143658810325477,
 8.5: 1.8294051627384962,
 9.0: 1.43658810325477,
 9.5: 1.0886644219977553,
 10.0: 0.8978675645342313,
 10.5: 0.6734006734006733,
 11.0: 0.47138047138047134,
 11.5: 0.25813692480359146}

In [236]:
thres = px.line(x=list(fric_dic.keys()), y=list(fric_dic.values()), title='Friction (%) against hours.', labels={'x': 'Threshold in hours', 'y': 'Percentage of friction'})
thres.add_vline(x=1.5, line_color='red')
thres.add_annotation(x=1.4, y=max(fric_dic.values()) , text='Mean delay', textangle=-90, showarrow=False)
thres.show()

Whereas the threshold of 1.5h seems to efficiently reduce the amount of friction among delayed end-of-rental, the window could be considered to up to ~3h, above which the feature efficiency would be less significant

# Scope: should we enable the feature for all cars?, only Connect cars?

In [132]:
px.histogram(data_late, 'delay_at_checkout_in_minutes_converted_in_hours', color='checkin_type', color_discrete_map={'mobile': 'lightblue', 'connect': 'purple'})

Given the small proportion of cars rented through the use of Connect in comparison of mobile, it is not recommended to activate the feature only on cars with théconnec tfeature

## Which share of our owner’s revenue would potentially be affected by the feature?


Given a 1.5h threshold, all rentals that occured with a delta < 1.5h (90 min) would not have been possible, which would represent an absolute loss of revenue for Getaround. Be mindful to keep the missing delta value in, as they represent a delta greater than 12h (and therefore no friction !), which amount to 91% of the data !!!

In [149]:
na_per(data_late)

No missing value in the column 'rental_id'.
No missing value in the column 'car_id'.
No missing value in the column 'checkin_type'.
No missing value in the column 'state'.
No missing value in the column 'delay_at_checkout_in_minutes'.
91.37% of data is missing in the column 'previous_ended_rental_id'.
91.37% of data is missing in the column 'time_delta_with_previous_rental_in_minutes'.
No missing value in the column 'delay_at_checkout_in_minutes_converted_in_hours'.
91.37% of data is missing in the column 'time_delta_with_previous_rental_in_minutes_converted_in_hours'.


In [166]:
# data_delta_thres = the remaining rentals if the feature was enabled on all cars (mobile + connect)
delta_thres = data_late['time_delta_with_previous_rental_in_minutes_converted_in_hours'] >= 1.5
delta_na = data_late['time_delta_with_previous_rental_in_minutes_converted_in_hours'].isna()
data_delta_thres = data_late[delta_thres | delta_na]


In [168]:
data_delta_thres.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8650 entries, 2 to 21309
Data columns (total 9 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   rental_id                                                      8650 non-null   int64  
 1   car_id                                                         8650 non-null   int64  
 2   checkin_type                                                   8650 non-null   object 
 3   state                                                          8650 non-null   object 
 4   delay_at_checkout_in_minutes                                   8650 non-null   float64
 5   previous_ended_rental_id                                       509 non-null    float64
 6   time_delta_with_previous_rental_in_minutes                     509 non-null    float64
 7   delay_at_checkout_in_minutes_converted_in_hours                86

In [169]:
data_delta_thres['time_delta_with_previous_rental_in_minutes_converted_in_hours'].isna().value_counts()

time_delta_with_previous_rental_in_minutes_converted_in_hours
True     8141
False     509
Name: count, dtype: int64

In [180]:
pot_loss = len(data_late) - len(data_delta_thres)
pot_loss_per = pot_loss / len(data_late)
print(f'The potential loss with current threshold impacts {pot_loss_per * 100:.2f} % ({pot_loss} rentals) of the recorded rentals.')

The potential loss with current threshold impacts 2.92 % (260 rentals) of the recorded rentals.


In [176]:
data_price.head()

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


Unfortunately, there is no car_id in the data_price df, so impossible to calculate the true revenue loss per day. But we can still apply the pot_loss_percentage to the total revenue per day (**assuming** that data_delay is representing all rental per day,, but we do not have this info), with some adjustements !

Indeed, pot loss percentage was calculated based only **on late drivers** (57 %, calculated above), whereas sum of revenue per day is on total drivers. We need to adjust the sum of revenue per day to account for this.

In [179]:
price_day = data_price['rental_price_per_day'].sum()
print(f'The total revenue per day (assumed) is {price_day:,}$')

The total revenue per day (assumed) is 586,731$


In [183]:
pot_loss_per

0.029180695847362513

In [193]:
# Adjustmed revenue and potential loss

late_per = 0.57

price_day_late = price_day*late_per
price_day_late_adjust = price_day_late - (price_day_late * pot_loss_per)
price_day_adjust = price_day_late_adjust + (price_day*(1-late_per))

print(f'The total revenue per day (assumed) with the feature activated would be of {price_day_adjust:,.2f}$, which corresponds to an absolute loss of {price_day-price_day_adjust:,.2f}$ ({(price_day-price_day_adjust)/price_day*100:.2f} %) per day')



The total revenue per day (assumed) with the feature activated would be of 576,971.91$, which corresponds to an absolute loss of 9,759.09$ (1.66 %) per day


Here we have it ! Activating the feature across all cars (mobile + connect) with a threshold of 1.5h (90 min) would cost **~9,800 $** (representing a **loss of ~1.7 %**) per day, but potentially represents a **significant increase of client satisfaction** !

## How many rentals would be affected by the feature depending on the threshold and scope we choose?

This was already done above, but we can make it more scalable through the definition of a function, which will be useful for deploying the dashboard later.

This will include calculation for both 

1) the dataset trimmed for late drivers only

2) the entire dataset

as both are informative and needed for further analysis

In [250]:
def trim_thres_late(i):
    '''Will trim the dataset of late drivers based on defined threshold and return potential loss in absolute and percentage value as a tuple '''

    delta_thres = data_late['time_delta_with_previous_rental_in_minutes_converted_in_hours'] >= i
    delta_na = data_late['time_delta_with_previous_rental_in_minutes_converted_in_hours'].isna()
    data_delta_thres = data_late[delta_thres | delta_na]
    pot_loss = len(data_late) - len(data_delta_thres)
    pot_loss_per = pot_loss / len(data_late)
    print(f'The potential loss with a threshold impacts {pot_loss_per * 100:.2f} % ({pot_loss} rentals) of the recorded rentals that were late.')
    return pot_loss, pot_loss_per

In [238]:
pot_loss, pot_loss_per = trim_thres_late(1.5)

The potential loss with a threshold impacts 2.92 % (260 rentals) of the recorded rentals that were late.


In [239]:
# Allows to easily calculate the potential loss with the upper range of the possible threshold window defined above with the decreasing friction plot
pot_loss, pot_loss_per = trim_thres_late(3)

The potential loss with a threshold impacts 4.35 % (388 rentals) of the recorded rentals that were late.


Here is the version for the **entire** dataset, and not just the late drivers

In [242]:
def trim_thres(i):
    '''Will trim the entire dataset of drivers based on defined threshold and return potential loss in absolute and percentage value as a tuple '''

    delta_tot = data_delay['time_delta_with_previous_rental_in_minutes_converted_in_hours'] >= i
    delta_na_tot = data_delay['time_delta_with_previous_rental_in_minutes_converted_in_hours'].isna()
    data_delay_thres = data_delay[delta_tot | delta_na_tot]
    pot_loss_tot = len(data_delay) - len(data_delay_thres)
    pot_loss_per_tot = pot_loss_tot / len(data_delay)
    print(f'The potential loss with a threshold impacts {pot_loss_per_tot * 100:.2f} % ({pot_loss_tot} rentals) of all the recorded rentals (late or not).')

In [243]:
trim_thres(1.5)

The potential loss with a threshold impacts 1.82 % (584 rentals) of all the recorded rentals (late or not).


In [244]:
trim_thres(3)

The potential loss with a threshold impacts 1.82 % (870 rentals) of all the recorded rentals (late or not).


Same as above, I defined function to make it more scalable for the futur deployment of dashboard

In [247]:
def revenue_loss(pot_loss_per):
    '''Will estimate the potential revenue loss per day based on result from the function trim_thres_late() and return loss and adjusted loss per day (relative to the entire dataset, 
    late or not, corrected for the % of late drivers) as a tuple'''

    late_per = len(data_late) / len(data_delay_trim)
    price_day = data_price['rental_price_per_day'].sum()
    price_day_late = price_day*late_per
    price_day_late_adjust = price_day_late - (price_day_late * pot_loss_per)
    price_day_adjust = price_day_late_adjust + (price_day*(1-late_per))
    print(f'The total revenue per day (assumed) with the feature activated would be of {price_day_adjust:,.2f}$, which corresponds to an absolute loss of {price_day-price_day_adjust:,.2f}$ ({(price_day-price_day_adjust)/price_day*100:.2f} %) per day')
    return price_day, price_day_adjust, late_per


In [251]:
# Ensure that we calculate everything based on the defined 1.5h threshold
pot_loss, pot_loss_per = trim_thres_late(1.5)

The potential loss with a threshold impacts 2.92 % (260 rentals) of the recorded rentals that were late.


In [252]:
price_day, price_day_adjust, late_per = revenue_loss(pot_loss_per)

The total revenue per day (assumed) with the feature activated would be of 576,957.16$, which corresponds to an absolute loss of 9,773.84$ (1.67 %) per day


## How often are drivers late for the next check-in? How does it impact the next driver?


Already discussed above = 57% of drivers are late !

In [290]:
len(data_late) / len(data_delay_trim)

0.5708610968733983

## How many problematic cases will it solve depending on the chosen threshold and scope?


To answer this question, we need to calculate, in data_delay, the amount of rentals for which **data_late is < threshold** among late drivers, which is the opposite of what the friction() function does (calculate the reminaing friction after threshold)


In [281]:
nb_prob, late_percentage = friction(1.5)

There was 31.01% of friction (2763 problematic cases) corrected with a 1.5h threshold.


In [282]:
nb_prob, late_percentage = friction(3)

There was 14.16% of friction (1262 problematic cases) corrected with a 3h threshold.


In [287]:
def friction_solved(threshold):
    '''Will calculate the number of problematic cases (friction) solved among late drivers after defining threshold '''

    friction_solved = threshold > data_late['delay_at_checkout_in_minutes_converted_in_hours']
    nb_prob_solved = friction_solved.sum()
    late_percentage_solved = nb_prob_solved/len(data_late)
    print(f'There was {late_percentage_solved*100:.2f}% of friction_solved ({nb_prob_solved} problematic cases) corrected with a {threshold}h threshold.')
    return nb_prob_solved, late_percentage_solved

In [288]:
nb_prob_solved, late_percentage_solved = friction_solved(1.5)

There was 68.61% of friction_solved (6113 problematic cases) corrected with a 1.5h threshold.


In [289]:
nb_prob_solved, late_percentage_solved = friction_solved(3)

There was 85.72% of friction_solved (7638 problematic cases) corrected with a 3h threshold.
